#### Installations

In [2]:
!nvidia-smi

Sat Jun  8 08:53:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:02:00.0 Off |                  Off |
|  0%   40C    P8             18W /  450W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%%capture

!pip install transformers datasets evaluate accelerate

#### Imports

In [1]:
import math
from tqdm.auto import tqdm
import numpy as np
import os

from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed

import torch
from torch.utils.data import DataLoader

import evaluate

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig,
    DataCollatorForSeq2Seq, get_scheduler, T5ForConditionalGeneration,
)

from datasets import load_dataset

#### Hyperparamters

In [19]:
SEED = 42

dataset_name = 'scan'
dataset_config_name = 'simple'
trust_remote_code = True
preprocessing_num_workers = None
overwrite_cache = False

gradient_accumulation_steps = 1

model_name = 'google-t5/t5-base'
#model_name = 'google/flan-t5-base'
# gpt2
# gemma

source_prefix = ""
max_target_length = 1024
max_source_length = 1024
padding = False
ignore_pad_token_for_loss = True
per_device_train_batch_size = 16
per_device_eval_batch_size = 16

weight_decay = 0.0
learning_rate = 5e-5
train_steps = 100000
eval_steps = 4000
lr_scheduler_type = 'linear'
num_warmup_steps = 0
checkpointing_steps = None
num_beams = 1

#output_dir = './'
output_dir = '/users/ujan/caricatures/models/scan_t5_base/'

#### Setup accelerator

In [12]:
accelerator = Accelerator(gradient_accumulation_steps=gradient_accumulation_steps)
set_seed(SEED)

#### Get dataset

In [20]:
raw_datasets = load_dataset(dataset_name, dataset_config_name, trust_remote_code=trust_remote_code)

#### Split train set into train and validation

In [21]:
train_val_split = raw_datasets['train'].train_test_split(test_size=0.1, seed=SEED)
raw_datasets['train'] = train_val_split['train']
raw_datasets['validation'] = train_val_split['test']

#### Load pretrained model or initialize from scratch. Load tokenizer

In [7]:
config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)

# pretrained model
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config, trust_remote_code=trust_remote_code)

/Users/ujan/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# from scratch
model = T5ForConditionalGeneration(config=config)
generation_config = GenerationConfig.from_pretrained(model_name)

#### Resize the embeddings when necessary to avoid index errors

In [25]:
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
  model.resize_token_embeddings(len(tokenizer))
if model.config.decoder_start_token_id is None:
  raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

prefix = source_prefix

#### Preprocess dataset

In [26]:
column_names = raw_datasets["train"].column_names
input_column = column_names[0]
output_column = column_names[1]


def preprocess_function(examples):
  inputs = examples[input_column]
  targets = examples[output_column]
  inputs = [prefix + inp for inp in inputs]
  model_inputs = tokenizer(
      inputs, max_length=max_source_length, padding=padding, truncation=True)

  # tokenize targets with the `text_target` keyword argument
  labels = tokenizer(text_target=targets, max_length=max_target_length,
                     padding=padding, truncation=True)

  # if we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
  # padding in the loss.
  if padding == "max_length" and ignore_pad_token_for_loss:
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [27]:
with accelerator.main_process_first():
  train_dataset = raw_datasets["train"].map(
      preprocess_function,
      batched=True,
      num_proc=preprocessing_num_workers,
      remove_columns=column_names,
      load_from_cache_file=not overwrite_cache,
      desc="Running tokenizer on dataset",
  )

  eval_dataset = raw_datasets["validation"].map(
      preprocess_function,
      batched=True,
      num_proc=preprocessing_num_workers,
      remove_columns=column_names,
      load_from_cache_file=not overwrite_cache,
      desc="Running tokenizer on dataset",
  )


Running tokenizer on dataset:   0%|          | 0/1673 [00:00<?, ? examples/s]

#### Data collator

In [28]:
label_pad_token_id = -100 if ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if accelerator.use_fp16 else None,
)

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=per_device_train_batch_size
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=per_device_eval_batch_size)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:593: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(


#### Optimizer

In [29]:
# split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)

#### Scheduler

In [30]:
overrode_max_train_steps = False

lr_scheduler = get_scheduler(
    name=lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps * accelerator.num_processes,
    num_training_steps=train_steps * accelerator.num_processes,
)

#### Prepare everything with Accelerate

In [31]:
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

# Train

In [ ]:
progress_bar = tqdm(range(train_steps), disable=not accelerator.is_local_main_process)
eval_bar = tqdm(range(len(eval_dataloader)), disable=not accelerator.is_local_main_process)

total_loss = 0
completed_steps = 0
model.train()

while True:
  for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
      outputs = model(**batch)
      loss = outputs.loss
      total_loss += loss.detach().float()
      accelerator.backward(loss)
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      # checks if the accelerator has performed an optimization step behind the scenes
      if accelerator.sync_gradients:
        progress_bar.update(1)
        completed_steps += 1

      if completed_steps % eval_steps == 0:
        model.eval()

        # metric
        accuracy = 0.0

        gen_kwargs = {
            "max_length": max_target_length,
            "num_beams": num_beams,
        }
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
              generated_tokens = accelerator.unwrap_model(model).generate(
                  batch["input_ids"],
                  attention_mask=batch["attention_mask"],
                  generation_config=generation_config,
                  **gen_kwargs,
              )

              generated_tokens = accelerator.pad_across_processes(
                  generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
                )
              labels = batch["labels"]
              # we did not pad to max length, we need to pad the labels too
              labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

              generated_tokens, labels = accelerator.gather_for_metrics((generated_tokens, labels))
              generated_tokens = generated_tokens.cpu().numpy()
              labels = labels.cpu().numpy()

              if ignore_pad_token_for_loss:
                # replace -100 in the labels as we can't decode them.
                labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
              if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
              decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
              decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

              accuracy += sum([decoded_preds[i] == decoded_labels[i] for i in range(len(decoded_preds))])

              eval_bar.update(1)

        eval_bar.refresh()
        eval_bar.reset()

        accelerator.print('steps : {}'.format(completed_steps))
        accelerator.print('accuracy : {}'.format(accuracy / len(raw_datasets['validation'])))
        accelerator.print('')

        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        # save
        new_path = output_dir+'checkpoint_'+str(completed_steps)
        if not os.path.isdir(new_path):
          os.makedirs(new_path)
        unwrapped_model.save_pretrained(
            new_path, is_main_process=accelerator.is_main_process, save_function=accelerator.save
        )

        model.train()

    if completed_steps == train_steps:
      raise

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

steps : 4000
accuracy : 0.03765690376569038



#### Test

In [9]:
test_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir+'checkpoint_18000', config=config)

In [19]:
test_dataset = raw_datasets["test"].map(
      preprocess_function,
      batched=True,
      num_proc=preprocessing_num_workers,
      remove_columns=column_names,
      load_from_cache_file=not overwrite_cache,
      desc="Running tokenizer on dataset",
  )

In [14]:
label_pad_token_id = -100 if ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if accelerator.use_fp16 else None,
)

test_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=per_device_eval_batch_size)

/Users/ujan/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/accelerate/accelerator.py:538: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(


In [15]:
test_model, test_dataloader = accelerator.prepare(test_model, test_dataloader)

In [16]:
test_bar = tqdm(range(len(test_dataloader)), disable=not accelerator.is_local_main_process)

test_model.eval()

# metric
accuracy = 0.0

gen_kwargs = {
    "max_length": max_target_length,
    "num_beams": num_beams,
}
for step, batch in enumerate(test_dataloader):
    with torch.no_grad():
      generated_tokens = accelerator.unwrap_model(test_model).generate(
          batch["input_ids"],
          attention_mask=batch["attention_mask"],
          **gen_kwargs,
      )

      generated_tokens = accelerator.pad_across_processes(
          generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
      labels = batch["labels"]
      # we did not pad to max length, we need to pad the labels too
      labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

      generated_tokens, labels = accelerator.gather_for_metrics((generated_tokens, labels))
      generated_tokens = generated_tokens.cpu().numpy()
      labels = labels.cpu().numpy()

      if ignore_pad_token_for_loss:
        # replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
      if isinstance(generated_tokens, tuple):
        generated_tokens = generated_tokens[0]
      decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

      accuracy += sum([decoded_preds[i] == decoded_labels[i] for i in range(len(decoded_preds))])

      test_bar.update(1)

accelerator.print('accuracy : {}'.format(accuracy / len(raw_datasets['test'])))

  0%|          | 0/523 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 18k accuracy (pre-trained) : 0.9791965566714491


#### Sample max lengths

In [22]:
# simple -> 14
# length -> 14

maxlen = -1
for sample in raw_datasets['train']:
    model_inputs = tokenizer(sample['commands'], truncation=False)
    l = len(model_inputs['input_ids'])
    if l > maxlen:
        maxlen = l

maxlen

14

In [23]:
# simple -> 265
# length -> train: 141, test: 265

maxlen = -1
for sample in raw_datasets['test']:
    model_outputs = tokenizer(sample['actions'], truncation=False)
    l = len(model_outputs['input_ids'])
    if l > maxlen:
        maxlen = l

maxlen

265

In [87]:
maxlen

265